### First-Class Objects
In Python, functions are first-class objects. This means that functions can be passed around and used as arguments, just like any other object (string, int, float, list, and so on). 

In [2]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

In [3]:
greet_bob(say_hello)

'Hello Bob'

In [4]:
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'

### Inner Functions
It’s possible to define functions inside other functions. Such functions are called inner functions. 

In [6]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()

In [7]:
parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


### Returning Functions From Functions
Python also allows you to use functions as return values. The following example returns one of the inner functions from the outer parent() function:

In [10]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

In [12]:
first = parent(1)
second = parent(2)
print(first)
print(second)

<function parent.<locals>.first_child at 0x058EEB70>
<function parent.<locals>.second_child at 0x058EEC00>


In [13]:
first()

'Hi, I am Emma'

In [14]:
second()

'Call me Liam'

### Simple Decorators

Decorators wrap a function, modifying its behavior.

In [18]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

print(say_whee)
# So-called 'decoration'
say_whee = my_decorator(say_whee)
print(say_whee)

<function say_whee at 0x058B4150>
<function my_decorator.<locals>.wrapper at 0x058B4300>


In [17]:
# What will happen?
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


In [19]:
#  the following example will only run the decorated code during the day
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def say_whee():
    print("Whee!")

say_whee = not_during_the_night(say_whee)

In [20]:
# So if you try to call say_whee() after bedtime, nothing will happen
say_whee()

### Syntactic Sugar

Python allows you to use decorators in a simpler way with the @ symbol, sometimes called the “pie” syntax.

In [22]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper


#  @my_decorator is just an easier way of saying say_whee = my_decorator(say_whee).
@my_decorator
def say_whee():
    print("Whee!")
    
print(say_whee)

<function my_decorator.<locals>.wrapper at 0x058EEE88>


### Decorating Functions With Arguments

In [32]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

In [34]:
# As the current decorator doesn't accept arguments, this will fail.
greet('hello')

TypeError: wrapper_do_twice() takes 0 positional arguments but 1 was given

The solution is to use args and ***kwargs* in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments.

In [43]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        """This is a wrapper."""
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def greet(name):
    """Say greetings.
    
    Args:
      name: A string.
    """
    print(f"Hello {name}")

In [38]:
greet('hello')

Hello hello
Hello hello


In [44]:
help(greet)

Help on function wrapper_do_twice in module __main__:

wrapper_do_twice(*args, **kwargs)
    This is a wrapper.



In [48]:
# To fix this, use functools.warps
# The @functools.wraps decorator uses the function functools.update_wrapper() 
# to update special attributes like __name__ and __doc__ that are used in the introspection.
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        """This is a wrapper."""
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def greet(name):
    """Say greetings.
    
    Args:
      name: A string.
    """
    print(f"Hello {name}")

In [47]:
help(greet)

Help on function greet in module __main__:

greet(name)
    Say greetings.
    
    Args:
      name: A string.



### Decorators With Arguments
Add another outer function to existing function...

In [53]:
import functools

def repeat(number):
    def repeat_wrapper(func):
        @functools.wraps(func)
        def wrapper_do_twice(*args, **kwargs):
            """This is a wrapper."""
            for _ in range(number):
                func(*args, **kwargs)
        return wrapper_do_twice
    return repeat_wrapper

@repeat(number=3)
def greet(name):
    """Say greetings.
    
    Args:
      name: A string.
    """
    print(f"Hello {name}")

In [54]:
greet('hello')

Hello hello
Hello hello
Hello hello


### boilerplate templates 

In [49]:
# This formula is a good boilerplate template for building more complex decorators.
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

In [50]:
# A @timer decorator. 
# It will measure the time a function takes to execute and print the duration to the console.
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [ ]:
# User authentication.
from flask import Flask, g, request, redirect, url_for
import functools
app = Flask(__name__)

def login_required(func):
    """Make sure user is logged in before proceeding"""
    @functools.wraps(func)
    def wrapper_login_required(*args, **kwargs):
        if g.user is None:
            return redirect(url_for("login", next=request.url))
        return func(*args, **kwargs)
    return wrapper_login_required

@app.route("/secret")
@login_required
def secret():
    ...